실습 예제  
--------------
## 1.	data_concrete.csv 를 사용한 콩크리트의 강도 문제를 (선형회귀)를 TensorFlow로 풀으시오 (주의 Keras 아님!!!). strength ~ . 와 같은 수식을 적용하시오.  

#### a.	설명변수를 정규화하여 전처리 하시오.  
#### b.	3개의 노드로 이루어진 하나의 은닉층을 만드시오.  
#### c.	수치예측 (회귀)에 적합한 출력층 구조를 만드시오.  
#### d.	In-Sample 테스팅한 결과를 MSE와 RMSE로 정리하여 보시오.  





In [156]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [242]:
## 텐서보드 쥬피터 안에 그리기
from IPython.display import clear_output, Image, display, HTML
import numpy as np
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add()
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
<script>
function load() {{
document.getElementById("{id}").pbtxt = {data};
}}
</script>
<link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
<div style="height:600px">
<tf-graph-basic id="{id}"></tf-graph-basic>
</div>
""".format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
    iframe = """
<iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
""".format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [157]:
data_origin = pd.read_csv("test_data\data_concrete.csv", engine='python')
data_origin.head()

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [158]:
data = data_origin.drop(["strength"], axis=1)
labels = data_origin["strength"].values

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 8 columns):
cement          1030 non-null float64
slag            1030 non-null float64
ash             1030 non-null float64
water           1030 non-null float64
superplastic    1030 non-null float64
coarseagg       1030 non-null float64
fineagg         1030 non-null float64
age             1030 non-null int64
dtypes: float64(7), int64(1)
memory usage: 64.5 KB


In [159]:
data.describe()

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000


원래는 트레인 데이터와 테스트 데이터를 나눠야 하지만 이번 문제에서는 생략한다.

## a.	설명변수를 정규화하여 전처리 하시오.  

In [161]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('minmax_scaler', MinMaxScaler()),
    ('std_scaler', StandardScaler()),
])



data_prepared = pipeline.fit_transform(data)

tmp = pd.DataFrame(data_prepared, columns=data.columns)
tmp.describe()

C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,cement,slag,ash,water,superplastic,coarseagg,fineagg,age
count,1.030000e+03,1.030000e+03,1.030000e+03,1.030000e+03,1.030000e+03,1.030000e+03,1.030000e+03,1.030000e+03
mean,6.898473e-17,-2.069542e-17,-8.278168e-17,3.587206e-16,2.207511e-16,2.690405e-16,-1.517664e-16,9.657862e-17
std,1.000486e+00,1.000486e+00,1.000486e+00,1.000486e+00,1.000486e+00,1.000486e+00,1.000486e+00,1.000486e+00
min,-1.715253e+00,-8.568879e-01,-8.471439e-01,-2.800211e+00,-1.039143e+00,-2.212138e+00,-2.240917e+00,-7.073594e-01
25%,-8.500535e-01,-8.568879e-01,-8.471439e-01,-7.808939e-01,-1.039143e+00,-5.265174e-01,-5.319697e-01,-6.123314e-01
50%,-7.915193e-02,-6.017783e-01,-8.471439e-01,1.608294e-01,3.271508e-02,-6.329352e-02,7.386739e-02,-2.797331e-01
75%,6.589606e-01,8.007446e-01,1.002278e+00,4.887927e-01,6.691307e-01,7.267605e-01,6.291661e-01,1.637312e-01
max,2.477915e+00,3.310675e+00,2.281084e+00,3.065647e+00,4.353642e+00,2.214224e+00,2.733062e+00,5.057677e+00


## b.	3개의 노드로 이루어진 하나의 은닉층을 만드시오.  

In [80]:
labels.reshape(-1,1)

array([[79.99],
       [61.89],
       [40.27],
       ...,
       [23.7 ],
       [32.77],
       [32.4 ]])

In [51]:
n_inputs = 8
n_hidden1 = 3
n_outputs = 1

In [75]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')

In [76]:
hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", 
                          activation=tf.nn.relu)

## c.	수치예측 (회귀)에 적합한 출력층 구조를 만드시오.  

회귀에는 linear 활성화 함수가 적합하다고 생각한다.

In [77]:
output = tf.layers.dense(hidden1, n_outputs, name="outputs",
                         activation=None, use_bias=True)

## d.	In-Sample 테스팅한 결과를 MSE와 RMSE로 정리하여 보시오.  

loss 함수와 optimizer함수를 정의하자

In [78]:
loss = tf.reduce_mean(tf.square(output-y))
loss2 = tf.sqrt(loss)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(loss)

train을 하자

In [81]:
init = tf.global_variables_initializer()

n_epochs = 1000

with tf.Session() as sess:
    init.run()
    for i in range(n_epochs):
        sess.run(train, feed_dict={X: data_prepared, y: labels.reshape(-1,1)})
    
    y_pred = output.eval(feed_dict={X: data_prepared})
    final_mse = mean_squared_error(labels, y_pred)
    final_rmse = np.sqrt(final_mse)
    print("final ---- mse, rmse = %s, %s" %(final_mse, final_rmse))
    print(y_pred.shape)

final ---- mse, rmse = 101.45789835386563, 10.072631153470558
(1030, 1)


In [84]:
display(pd.DataFrame(labels, columns=['labels']).describe())
display(pd.DataFrame(y_pred, columns=['y_pred']).describe())

,labels
count,1030.000000
mean,35.817961
std,16.705742
min,2.330000
25%,23.710000
50%,34.445000
75%,46.135000
max,82.600000


,y_pred
count,1030.000000
mean,35.848682
std,13.120500
min,11.513258
25%,25.434429
50%,33.356665
75%,46.054671
max,79.011803


기본 선형회귀의 rmse와 mse비교

In [85]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(data_prepared, labels)

train_predictions = lin_reg.predict(data_prepared)
lin_mse = mean_squared_error(labels, train_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_mse, lin_rmse

(107.19723607486019, 10.353609808895648)

In [86]:
display(pd.DataFrame(labels).describe())
display(pd.DataFrame(train_predictions, columns=['y_pred_linreg']).describe())

,0
count,1030.000000
mean,35.817961
std,16.705742
min,2.330000
25%,23.710000
50%,34.445000
75%,46.135000
max,82.600000


,y_pred_linreg
count,1030.000000
mean,35.817961
std,13.106502
min,10.455766
25%,25.995461
50%,33.457700
75%,45.102870
max,78.990876


비슷비슷한 결과가 나온다.

In [87]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(3,input_dim=8, activation='relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error',
                  optimizer='sgd')

model.fit(data_prepared, labels, epochs= 1000, batch_size=50)

y_predictions = model.predict(data_prepared).flatten()

Epoch 1/1000
1030/1030 [==============================] - 0s 127us/step - loss: 45.8425
Epoch 2/1000
1030/1030 [==============================] - 0s 70us/step - loss: 49.5955
Epoch 3/1000
1030/1030 [==============================] - 0s 63us/step - loss: 50.1053
Epoch 4/1000
1030/1030 [==============================] - 0s 61us/step - loss: 46.2518
Epoch 5/1000
1030/1030 [==============================] - 0s 66us/step - loss: 47.1919
Epoch 6/1000
1030/1030 [==============================] - 0s 61us/step - loss: 44.8977
Epoch 7/1000
1030/1030 [==============================] - 0s 62us/step - loss: 48.8356
Epoch 8/1000
1030/1030 [==============================] - 0s 62us/step - loss: 48.5570
Epoch 9/1000
1030/1030 [==============================] - 0s 72us/step - loss: 45.8216
Epoch 10/1000
1030/1030 [==============================] - 0s 63us/step - loss: 45.1999
Epoch 11/1000
1030/1030 [==============================] - 0s 62us/step - loss: 45.5708
Epoch 12/1000
1030/1030 [===============

1030/1030 [==============================] - 0s 70us/step - loss: 45.9441
Epoch 95/1000
1030/1030 [==============================] - 0s 67us/step - loss: 46.3414
Epoch 96/1000
1030/1030 [==============================] - 0s 64us/step - loss: 46.2248
Epoch 97/1000
1030/1030 [==============================] - 0s 60us/step - loss: 46.7851
Epoch 98/1000
1030/1030 [==============================] - 0s 60us/step - loss: 44.1832
Epoch 99/1000
1030/1030 [==============================] - 0s 64us/step - loss: 47.1224
Epoch 100/1000
1030/1030 [==============================] - 0s 67us/step - loss: 45.7360
Epoch 101/1000
1030/1030 [==============================] - 0s 67us/step - loss: 47.2483
Epoch 102/1000
1030/1030 [==============================] - 0s 59us/step - loss: 53.9100
Epoch 103/1000
1030/1030 [==============================] - 0s 58us/step - loss: 46.9807
Epoch 104/1000
1030/1030 [==============================] - 0s 59us/step - loss: 44.8912
Epoch 105/1000
1030/1030 [===============

1030/1030 [==============================] - 0s 64us/step - loss: 49.1049
Epoch 187/1000
1030/1030 [==============================] - 0s 59us/step - loss: 49.5016
Epoch 188/1000
1030/1030 [==============================] - 0s 57us/step - loss: 45.1403
Epoch 189/1000
1030/1030 [==============================] - 0s 58us/step - loss: 43.9277
Epoch 190/1000
1030/1030 [==============================] - 0s 59us/step - loss: 47.3586
Epoch 191/1000
1030/1030 [==============================] - 0s 58us/step - loss: 52.1836
Epoch 192/1000
1030/1030 [==============================] - 0s 54us/step - loss: 46.8865
Epoch 193/1000
1030/1030 [==============================] - 0s 60us/step - loss: 49.5361
Epoch 194/1000
1030/1030 [==============================] - 0s 67us/step - loss: 46.9289
Epoch 195/1000
1030/1030 [==============================] - 0s 68us/step - loss: 45.4894
Epoch 196/1000
1030/1030 [==============================] - 0s 70us/step - loss: 49.7216
Epoch 197/1000
1030/1030 [==========

1030/1030 [==============================] - 0s 83us/step - loss: 46.5562
Epoch 279/1000
1030/1030 [==============================] - 0s 74us/step - loss: 44.3990
Epoch 280/1000
1030/1030 [==============================] - 0s 74us/step - loss: 44.7368
Epoch 281/1000
1030/1030 [==============================] - 0s 58us/step - loss: 50.2847
Epoch 282/1000
1030/1030 [==============================] - 0s 54us/step - loss: 50.3210
Epoch 283/1000
1030/1030 [==============================] - 0s 68us/step - loss: 45.2184
Epoch 284/1000
1030/1030 [==============================] - 0s 73us/step - loss: 46.4044
Epoch 285/1000
1030/1030 [==============================] - 0s 66us/step - loss: 43.4296
Epoch 286/1000
1030/1030 [==============================] - 0s 68us/step - loss: 49.2600
Epoch 287/1000
1030/1030 [==============================] - 0s 61us/step - loss: 52.1782
Epoch 288/1000
1030/1030 [==============================] - 0s 64us/step - loss: 48.4297
Epoch 289/1000
1030/1030 [==========

1030/1030 [==============================] - 0s 60us/step - loss: 44.7440
Epoch 371/1000
1030/1030 [==============================] - 0s 62us/step - loss: 49.7260
Epoch 372/1000
1030/1030 [==============================] - 0s 58us/step - loss: 51.4931
Epoch 373/1000
1030/1030 [==============================] - 0s 58us/step - loss: 51.5223
Epoch 374/1000
1030/1030 [==============================] - 0s 57us/step - loss: 46.6749
Epoch 375/1000
1030/1030 [==============================] - 0s 59us/step - loss: 45.9282
Epoch 376/1000
1030/1030 [==============================] - 0s 63us/step - loss: 52.1471
Epoch 377/1000
1030/1030 [==============================] - 0s 58us/step - loss: 46.4419
Epoch 378/1000
1030/1030 [==============================] - 0s 59us/step - loss: 44.5740
Epoch 379/1000
1030/1030 [==============================] - 0s 60us/step - loss: 45.5621
Epoch 380/1000
1030/1030 [==============================] - 0s 61us/step - loss: 49.9364
Epoch 381/1000
1030/1030 [==========

...

1030/1030 [==============================] - 0s 60us/step - loss: 49.1030
Epoch 555/1000
1030/1030 [==============================] - 0s 57us/step - loss: 48.9043
Epoch 556/1000
1030/1030 [==============================] - 0s 54us/step - loss: 44.2456
Epoch 557/1000
1030/1030 [==============================] - 0s 52us/step - loss: 48.9392
Epoch 558/1000
1030/1030 [==============================] - 0s 54us/step - loss: 50.3983
Epoch 559/1000
1030/1030 [==============================] - 0s 56us/step - loss: 47.5056
Epoch 560/1000
1030/1030 [==============================] - 0s 56us/step - loss: 48.2020
Epoch 561/1000
1030/1030 [==============================] - 0s 55us/step - loss: 45.2710
Epoch 562/1000
1030/1030 [==============================] - 0s 53us/step - loss: 44.3354
Epoch 563/1000
1030/1030 [==============================] - 0s 56us/step - loss: 50.5217
Epoch 564/1000
1030/1030 [==============================] - 0s 52us/step - loss: 44.5622
Epoch 565/1000
1030/1030 [==========

1030/1030 [==============================] - 0s 64us/step - loss: 48.9700
Epoch 647/1000
1030/1030 [==============================] - 0s 63us/step - loss: 46.5742
Epoch 648/1000
1030/1030 [==============================] - 0s 65us/step - loss: 44.9399
Epoch 649/1000
1030/1030 [==============================] - 0s 62us/step - loss: 47.2863
Epoch 650/1000
1030/1030 [==============================] - 0s 57us/step - loss: 48.9933
Epoch 651/1000
1030/1030 [==============================] - 0s 59us/step - loss: 45.3926
Epoch 652/1000
1030/1030 [==============================] - 0s 55us/step - loss: 46.7454
Epoch 653/1000
1030/1030 [==============================] - 0s 59us/step - loss: 46.4969
Epoch 654/1000
1030/1030 [==============================] - 0s 58us/step - loss: 45.3703
Epoch 655/1000
1030/1030 [==============================] - 0s 59us/step - loss: 43.9764
Epoch 656/1000
1030/1030 [==============================] - 0s 59us/step - loss: 48.1931
Epoch 657/1000
1030/1030 [==========

1030/1030 [==============================] - 0s 58us/step - loss: 48.0465
Epoch 739/1000
1030/1030 [==============================] - 0s 65us/step - loss: 48.4490
Epoch 740/1000
1030/1030 [==============================] - 0s 58us/step - loss: 45.5834
Epoch 741/1000
1030/1030 [==============================] - 0s 61us/step - loss: 47.3410
Epoch 742/1000
1030/1030 [==============================] - 0s 59us/step - loss: 48.5226
Epoch 743/1000
1030/1030 [==============================] - 0s 58us/step - loss: 48.6211
Epoch 744/1000
1030/1030 [==============================] - 0s 60us/step - loss: 44.9815
Epoch 745/1000
1030/1030 [==============================] - 0s 58us/step - loss: 45.2950
Epoch 746/1000
1030/1030 [==============================] - 0s 57us/step - loss: 52.2243
Epoch 747/1000
1030/1030 [==============================] - 0s 56us/step - loss: 45.8223
Epoch 748/1000
1030/1030 [==============================] - 0s 60us/step - loss: 45.3860
Epoch 749/1000
1030/1030 [==========

1030/1030 [==============================] - 0s 80us/step - loss: 44.4442
Epoch 831/1000
1030/1030 [==============================] - 0s 97us/step - loss: 47.1295
Epoch 832/1000
1030/1030 [==============================] - 0s 89us/step - loss: 44.8238
Epoch 833/1000
1030/1030 [==============================] - 0s 82us/step - loss: 45.0533
Epoch 834/1000
1030/1030 [==============================] - 0s 62us/step - loss: 46.6580
Epoch 835/1000
1030/1030 [==============================] - 0s 64us/step - loss: 46.2248
Epoch 836/1000
1030/1030 [==============================] - 0s 60us/step - loss: 47.8582
Epoch 837/1000
1030/1030 [==============================] - 0s 67us/step - loss: 47.9405
Epoch 838/1000
1030/1030 [==============================] - 0s 61us/step - loss: 50.2519
Epoch 839/1000
1030/1030 [==============================] - 0s 65us/step - loss: 46.4487
Epoch 840/1000
1030/1030 [==============================] - 0s 62us/step - loss: 46.8978
Epoch 841/1000
1030/1030 [==========

1030/1030 [==============================] - 0s 60us/step - loss: 44.9457
Epoch 923/1000
1030/1030 [==============================] - 0s 63us/step - loss: 49.3924
Epoch 924/1000
1030/1030 [==============================] - 0s 56us/step - loss: 48.9144
Epoch 925/1000
1030/1030 [==============================] - 0s 57us/step - loss: 48.8456
Epoch 926/1000
1030/1030 [==============================] - 0s 57us/step - loss: 48.9423
Epoch 927/1000
1030/1030 [==============================] - 0s 56us/step - loss: 44.6001
Epoch 928/1000
1030/1030 [==============================] - 0s 52us/step - loss: 46.6885
Epoch 929/1000
1030/1030 [==============================] - 0s 53us/step - loss: 45.8666
Epoch 930/1000
1030/1030 [==============================] - 0s 56us/step - loss: 44.6916
Epoch 931/1000
1030/1030 [==============================] - 0s 58us/step - loss: 48.1814
Epoch 932/1000
1030/1030 [==============================] - 0s 58us/step - loss: 45.5193
Epoch 933/1000
1030/1030 [==========

In [88]:
display(pd.DataFrame(labels).describe())
display(pd.DataFrame(y_predictions, columns=['y_pred_keras']).describe())

,0
count,1030.000000
mean,35.817961
std,16.705742
min,2.330000
25%,23.710000
50%,34.445000
75%,46.135000
max,82.600000


,y_pred_keras
count,1030.000000
mean,36.284641
std,15.341105
min,-0.542231
25%,25.260380
50%,36.099155
75%,45.413254
max,75.589050


In [244]:
show_graph(tf.get_default_graph().as_graph_def())

## 2.	“data_spam.csv”를 가져오고 TensorFlow로 코딩하여 다음 물음에 답하여라. (설명 필수, 쥬피터 노트북 제출)   

In [325]:
data_origin = pd.read_csv("test_data\data_spam.csv", engine='python')
data_origin.head()

,email_id,is_spam,word_freq_will,word_freq_original,word_freq_415,word_freq_mail,char_freq_#,char_freq_$,word_freq_internet,word_freq_edu,...,word_freq_receive,word_freq_000,capital_run_length_average,word_freq_address,word_freq_george,word_freq_cs,word_freq_random,word_freq_conference,word_freq_technology,char_freq_(
0,3227,no,1.06,0.26,0.0,0.00,0.0,0.00,0.00,0.0,...,0.00,0.00,2.646,0.00,0.0,0.0,0,0.0,0.00,0.039
1,2331,no,0.00,0.00,0.0,1.40,0.0,0.00,0.00,0.0,...,0.00,0.00,3.875,0.00,1.4,0.0,0,0.0,0.00,0.220
2,1899,yes,0.42,0.00,0.0,0.42,0.0,0.39,0.00,0.0,...,0.42,3.38,26.405,0.00,0.0,0.0,0,0.0,0.42,0.097
3,1387,no,0.00,0.00,0.0,0.00,0.0,0.00,2.07,0.0,...,0.00,0.00,1.523,0.00,0.0,0.0,0,0.0,0.00,0.144
4,3555,no,1.94,0.00,0.0,0.00,0.0,0.00,0.00,0.0,...,0.00,0.00,1.197,0.27,0.0,0.0,0,0.0,0.00,0.000


In [326]:
data_origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 63 columns):
email_id                      1000 non-null int64
is_spam                       1000 non-null object
word_freq_will                1000 non-null float64
word_freq_original            1000 non-null float64
word_freq_415                 1000 non-null float64
word_freq_mail                1000 non-null float64
char_freq_#                   1000 non-null float64
char_freq_$                   1000 non-null float64
word_freq_internet            1000 non-null float64
word_freq_edu                 1000 non-null float64
word_freq_hp                  1000 non-null float64
word_freq_lab                 1000 non-null float64
char_freq_!                   1000 non-null float64
word_freq_forest              1000 non-null int64
word_freq_parts               1000 non-null float64
word_freq_order               1000 non-null float64
word_freq_credit              1000 non-null float64
word_freq_direc

In [327]:
data_origin["word_freq_cs"].value_counts()

0.0    965
Name: word_freq_cs, dtype: int64

결측치가 있는 특성이 모두 0 값이므로 그 특성을 버린다!

In [482]:
data = data_origin.drop(["email_id","word_freq_cs", "is_spam"], axis=1)
labels = data_origin['is_spam'].copy()

labels를 0,1로 치환하자

In [483]:
labels[labels=='no'] = 0
labels[labels=='yes'] = 1

## a.	데이터를 70% 학습용, 30% 시험용으로 분리 하여라.  

In [407]:
from sklearn.model_selection import train_test_split

train, test, train_labels, test_labels = train_test_split(data, labels, test_size=0.3, random_state=42)

## b.	로지스틱 회귀 분류 예측 모형을 만들어서 학습 및 예측을 하여라.  

In [408]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 700 entries, 541 to 102
Data columns (total 60 columns):
word_freq_will                700 non-null float64
word_freq_original            700 non-null float64
word_freq_415                 700 non-null float64
word_freq_mail                700 non-null float64
char_freq_#                   700 non-null float64
char_freq_$                   700 non-null float64
word_freq_internet            700 non-null float64
word_freq_edu                 700 non-null float64
word_freq_hp                  700 non-null float64
word_freq_lab                 700 non-null float64
char_freq_!                   700 non-null float64
word_freq_forest              700 non-null int64
word_freq_parts               700 non-null float64
word_freq_order               700 non-null float64
word_freq_credit              700 non-null float64
word_freq_direct              700 non-null float64
word_freq_project             700 non-null float64
word_freq_neural            

In [505]:
n_inputs = 60
n_hidden1 = 30
n_hidden2 = 30
n_outputs = 2

노드를 깔아보자

In [538]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [539]:
with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",
                             activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2",
                             activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

loss를 정의하자

In [540]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

optimizer 정의하기

In [541]:
learning_rate = 0.001

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training = optimizer.minimize(loss)

In [542]:
help(tf.as_dtype)

Help on function as_dtype in module tensorflow.python.framework.dtypes:

as_dtype(type_value)
    Converts the given `type_value` to a `DType`.
    
    Args:
      type_value: A value that can be converted to a `tf.DType` object. This may
        currently be a `tf.DType` object, a [`DataType`
        enum](https://www.tensorflow.org/code/tensorflow/core/framework/types.proto),
        a string type name, or a `numpy.dtype`.
    
    Returns:
      A `DType` corresponding to `type_value`.
    
    Raises:
      TypeError: If `type_value` cannot be converted to a `DType`.



In [543]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, tf.cast(y, tf.int32), 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [544]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [545]:
n_epochs = 9000

with tf.Session() as sess:
    init.run()
    
    for i in range(n_epochs):
        sess.run(training, feed_dict={X: train, y: train_labels})
        acc = accuracy.eval(feed_dict={X: train, y: train_labels})
        print("epoch:", i,"정확도: ", acc)
        
    save_path = saver.save(sess, "./my_model_final.ckpt")

epoch: 0 정확도:  0.66714287
epoch: 1 정확도:  0.3042857
epoch: 2 정확도:  0.6614286
epoch: 3 정확도:  0.28857142
epoch: 4 정확도:  0.6585714
epoch: 5 정확도:  0.2857143
epoch: 6 정확도:  0.6557143
epoch: 7 정확도:  0.28428572
epoch: 8 정확도:  0.6542857
epoch: 9 정확도:  0.28428572
epoch: 10 정확도:  0.64857143
epoch: 11 정확도:  0.28285715
epoch: 12 정확도:  0.64
epoch: 13 정확도:  0.28285715
epoch: 14 정확도:  0.63857144
epoch: 15 정확도:  0.28857142
epoch: 16 정확도:  0.63285714
epoch: 17 정확도:  0.2942857
epoch: 18 정확도:  0.63857144
epoch: 19 정확도:  0.29714286
epoch: 20 정확도:  0.64
epoch: 21 정확도:  0.29857144
epoch: 22 정확도:  0.63857144
epoch: 23 정확도:  0.29714286
epoch: 24 정확도:  0.63857144
epoch: 25 정확도:  0.29714286
epoch: 26 정확도:  0.6342857
epoch: 27 정확도:  0.3
epoch: 28 정확도:  0.62857145
epoch: 29 정확도:  0.30285713
epoch: 30 정확도:  0.62857145
epoch: 31 정확도:  0.30285713
epoch: 32 정확도:  0.63
epoch: 33 정확도:  0.30571428
epoch: 34 정확도:  0.62714285
epoch: 35 정확도:  0.30285713
epoch: 36 정확도:  0.62714285
epoch: 37 정확도:  0.30857143
epoch: 38 정확도:  0

epoch: 330 정확도:  0.7542857
epoch: 331 정확도:  0.7128571
epoch: 332 정확도:  0.75714284
epoch: 333 정확도:  0.70285714
epoch: 334 정확도:  0.76
epoch: 335 정확도:  0.70285714
epoch: 336 정확도:  0.7557143
epoch: 337 정확도:  0.70714283
epoch: 338 정확도:  0.7542857
epoch: 339 정확도:  0.7214286
epoch: 340 정확도:  0.75857145
epoch: 341 정확도:  0.71428573
epoch: 342 정확도:  0.7614286
epoch: 343 정확도:  0.71
epoch: 344 정확도:  0.7614286
epoch: 345 정확도:  0.70714283
epoch: 346 정확도:  0.7614286
epoch: 347 정확도:  0.7114286
epoch: 348 정확도:  0.75857145
epoch: 349 정확도:  0.7157143
epoch: 350 정확도:  0.75714284
epoch: 351 정확도:  0.7185714
epoch: 352 정확도:  0.76
epoch: 353 정확도:  0.7171429
epoch: 354 정확도:  0.76
epoch: 355 정확도:  0.72
epoch: 356 정확도:  0.76
epoch: 357 정확도:  0.7185714
epoch: 358 정확도:  0.7614286
epoch: 359 정확도:  0.7185714
epoch: 360 정확도:  0.76285714
epoch: 361 정확도:  0.7228571
epoch: 362 정확도:  0.76285714
epoch: 363 정확도:  0.7285714
epoch: 364 정확도:  0.7614286
epoch: 365 정확도:  0.7214286
epoch: 366 정확도:  0.76428574
epoch: 367 정확도:  0.

epoch: 670 정확도:  0.7771429
epoch: 671 정확도:  0.7828571
epoch: 672 정확도:  0.7771429
epoch: 673 정확도:  0.7842857
epoch: 674 정확도:  0.78
epoch: 675 정확도:  0.7814286
epoch: 676 정확도:  0.78
epoch: 677 정확도:  0.7842857
epoch: 678 정확도:  0.78
epoch: 679 정확도:  0.78
epoch: 680 정확도:  0.78
epoch: 681 정확도:  0.7814286
epoch: 682 정확도:  0.78
epoch: 683 정확도:  0.78
epoch: 684 정확도:  0.78
epoch: 685 정확도:  0.78571427
epoch: 686 정확도:  0.7757143
epoch: 687 정확도:  0.78571427
epoch: 688 정확도:  0.7785714
epoch: 689 정확도:  0.7842857
epoch: 690 정확도:  0.7785714
epoch: 691 정확도:  0.78571427
epoch: 692 정확도:  0.78
epoch: 693 정확도:  0.7785714
epoch: 694 정확도:  0.78
epoch: 695 정확도:  0.7814286
epoch: 696 정확도:  0.78
epoch: 697 정확도:  0.7814286
epoch: 698 정확도:  0.7814286
epoch: 699 정확도:  0.7842857
epoch: 700 정확도:  0.78
epoch: 701 정확도:  0.7785714
epoch: 702 정확도:  0.7785714
epoch: 703 정확도:  0.7842857
epoch: 704 정확도:  0.7785714
epoch: 705 정확도:  0.78
epoch: 706 정확도:  0.7828571
epoch: 707 정확도:  0.78571427
epoch: 708 정확도:  0.78
epoch: 709 정확

epoch: 1012 정확도:  0.7771429
epoch: 1013 정확도:  0.7871429
epoch: 1014 정확도:  0.7771429
epoch: 1015 정확도:  0.78571427
epoch: 1016 정확도:  0.78
epoch: 1017 정확도:  0.7885714
epoch: 1018 정확도:  0.78
epoch: 1019 정확도:  0.7885714
epoch: 1020 정확도:  0.7814286
epoch: 1021 정확도:  0.7871429
epoch: 1022 정확도:  0.7785714
epoch: 1023 정확도:  0.7871429
epoch: 1024 정확도:  0.7785714
epoch: 1025 정확도:  0.7871429
epoch: 1026 정확도:  0.78
epoch: 1027 정확도:  0.7885714
epoch: 1028 정확도:  0.7814286
epoch: 1029 정확도:  0.7885714
epoch: 1030 정확도:  0.7828571
epoch: 1031 정확도:  0.7885714
epoch: 1032 정확도:  0.78
epoch: 1033 정확도:  0.7885714
epoch: 1034 정확도:  0.7814286
epoch: 1035 정확도:  0.7871429
epoch: 1036 정확도:  0.78
epoch: 1037 정확도:  0.7871429
epoch: 1038 정확도:  0.7785714
epoch: 1039 정확도:  0.7871429
epoch: 1040 정확도:  0.78
epoch: 1041 정확도:  0.7885714
epoch: 1042 정확도:  0.7814286
epoch: 1043 정확도:  0.7871429
epoch: 1044 정확도:  0.78
epoch: 1045 정확도:  0.7871429
epoch: 1046 정확도:  0.78
epoch: 1047 정확도:  0.7871429
epoch: 1048 정확도:  0.78
epoch: 1

epoch: 1329 정확도:  0.80714285
epoch: 1330 정확도:  0.7885714
epoch: 1331 정확도:  0.80714285
epoch: 1332 정확도:  0.8
epoch: 1333 정확도:  0.81714284
epoch: 1334 정확도:  0.7871429
epoch: 1335 정확도:  0.78
epoch: 1336 정확도:  0.7871429
epoch: 1337 정확도:  0.80857146
epoch: 1338 정확도:  0.79571426
epoch: 1339 정확도:  0.8142857
epoch: 1340 정확도:  0.79714286
epoch: 1341 정확도:  0.8057143
epoch: 1342 정확도:  0.79
epoch: 1343 정확도:  0.81
epoch: 1344 정확도:  0.7871429
epoch: 1345 정확도:  0.79571426
epoch: 1346 정확도:  0.79
epoch: 1347 정확도:  0.80714285
epoch: 1348 정확도:  0.79
epoch: 1349 정확도:  0.81142855
epoch: 1350 정확도:  0.7885714
epoch: 1351 정확도:  0.7942857
epoch: 1352 정확도:  0.7885714
epoch: 1353 정확도:  0.8042857
epoch: 1354 정확도:  0.79142857
epoch: 1355 정확도:  0.80714285
epoch: 1356 정확도:  0.7985714
epoch: 1357 정확도:  0.8142857
epoch: 1358 정확도:  0.7814286
epoch: 1359 정확도:  0.7785714
epoch: 1360 정확도:  0.78571427
epoch: 1361 정확도:  0.8
epoch: 1362 정확도:  0.78
epoch: 1363 정확도:  0.80285716
epoch: 1364 정확도:  0.7871429
epoch: 1365 정확도:  0.7

...

epoch: 7627 정확도:  0.8314286
epoch: 7628 정확도:  0.8242857
epoch: 7629 정확도:  0.83428574
epoch: 7630 정확도:  0.83
epoch: 7631 정확도:  0.8314286
epoch: 7632 정확도:  0.83428574
epoch: 7633 정확도:  0.8457143
epoch: 7634 정확도:  0.7842857
epoch: 7635 정확도:  0.79571426
epoch: 7636 정확도:  0.82857144
epoch: 7637 정확도:  0.82857144
epoch: 7638 정확도:  0.8314286
epoch: 7639 정확도:  0.8357143
epoch: 7640 정확도:  0.8385714
epoch: 7641 정확도:  0.81857145
epoch: 7642 정확도:  0.82285714
epoch: 7643 정확도:  0.81714284
epoch: 7644 정확도:  0.82857144
epoch: 7645 정확도:  0.7757143
epoch: 7646 정확도:  0.79285717
epoch: 7647 정확도:  0.83
epoch: 7648 정확도:  0.8242857
epoch: 7649 정확도:  0.8357143
epoch: 7650 정확도:  0.8214286
epoch: 7651 정확도:  0.8385714
epoch: 7652 정확도:  0.8314286
epoch: 7653 정확도:  0.8485714
epoch: 7654 정확도:  0.77428573
epoch: 7655 정확도:  0.7757143
epoch: 7656 정확도:  0.82714283
epoch: 7657 정확도:  0.82714283
epoch: 7658 정확도:  0.82714283
epoch: 7659 정확도:  0.82857144
epoch: 7660 정확도:  0.8314286
epoch: 7661 정확도:  0.8385714
epoch: 7662 정확도

epoch: 7941 정확도:  0.83
epoch: 7942 정확도:  0.83285713
epoch: 7943 정확도:  0.8385714
epoch: 7944 정확도:  0.8428571
epoch: 7945 정확도:  0.8428571
epoch: 7946 정확도:  0.8585714
epoch: 7947 정확도:  0.82
epoch: 7948 정확도:  0.8142857
epoch: 7949 정확도:  0.82
epoch: 7950 정확도:  0.8214286
epoch: 7951 정확도:  0.77428573
epoch: 7952 정확도:  0.7814286
epoch: 7953 정확도:  0.82285714
epoch: 7954 정확도:  0.83285713
epoch: 7955 정확도:  0.83
epoch: 7956 정확도:  0.82714283
epoch: 7957 정확도:  0.8371429
epoch: 7958 정확도:  0.8414286
epoch: 7959 정확도:  0.8442857
epoch: 7960 정확도:  0.85571426
epoch: 7961 정확도:  0.8428571
epoch: 7962 정확도:  0.8457143
epoch: 7963 정확도:  0.75
epoch: 7964 정확도:  0.7171429
epoch: 7965 정확도:  0.80142856
epoch: 7966 정확도:  0.8157143
epoch: 7967 정확도:  0.8257143
epoch: 7968 정확도:  0.8242857
epoch: 7969 정확도:  0.82
epoch: 7970 정확도:  0.82714283
epoch: 7971 정확도:  0.8314286
epoch: 7972 정확도:  0.83285713
epoch: 7973 정확도:  0.83285713
epoch: 7974 정확도:  0.83285713
epoch: 7975 정확도:  0.8371429
epoch: 7976 정확도:  0.83428574
epoch: 797

epoch: 8265 정확도:  0.8371429
epoch: 8266 정확도:  0.8371429
epoch: 8267 정확도:  0.8471429
epoch: 8268 정확도:  0.8385714
epoch: 8269 정확도:  0.8585714
epoch: 8270 정확도:  0.7985714
epoch: 8271 정확도:  0.8042857
epoch: 8272 정확도:  0.8371429
epoch: 8273 정확도:  0.8414286
epoch: 8274 정확도:  0.8428571
epoch: 8275 정확도:  0.8485714
epoch: 8276 정확도:  0.8585714
epoch: 8277 정확도:  0.82714283
epoch: 8278 정확도:  0.82714283
epoch: 8279 정확도:  0.79142857
epoch: 8280 정확도:  0.79285717
epoch: 8281 정확도:  0.8314286
epoch: 8282 정확도:  0.84
epoch: 8283 정확도:  0.8414286
epoch: 8284 정확도:  0.8485714
epoch: 8285 정확도:  0.85714287
epoch: 8286 정확도:  0.81857145
epoch: 8287 정확도:  0.8142857
epoch: 8288 정확도:  0.8385714
epoch: 8289 정확도:  0.85714287
epoch: 8290 정확도:  0.81
epoch: 8291 정확도:  0.81
epoch: 8292 정확도:  0.8385714
epoch: 8293 정확도:  0.8428571
epoch: 8294 정확도:  0.8542857
epoch: 8295 정확도:  0.8314286
epoch: 8296 정확도:  0.8314286
epoch: 8297 정확도:  0.7657143
epoch: 8298 정확도:  0.76857144
epoch: 8299 정확도:  0.81
epoch: 8300 정확도:  0.8314286
epoc

epoch: 8596 정확도:  0.79714286
epoch: 8597 정확도:  0.83428574
epoch: 8598 정확도:  0.8371429
epoch: 8599 정확도:  0.8485714
epoch: 8600 정확도:  0.8457143
epoch: 8601 정확도:  0.8585714
epoch: 8602 정확도:  0.80857146
epoch: 8603 정확도:  0.81
epoch: 8604 정확도:  0.8385714
epoch: 8605 정확도:  0.8428571
epoch: 8606 정확도:  0.8485714
epoch: 8607 정확도:  0.8457143
epoch: 8608 정확도:  0.85714287
epoch: 8609 정확도:  0.7814286
epoch: 8610 정확도:  0.7785714
epoch: 8611 정확도:  0.8242857
epoch: 8612 정확도:  0.8357143
epoch: 8613 정확도:  0.83428574
epoch: 8614 정확도:  0.8371429
epoch: 8615 정확도:  0.8414286
epoch: 8616 정확도:  0.8485714
epoch: 8617 정확도:  0.85714287
epoch: 8618 정확도:  0.8357143
epoch: 8619 정확도:  0.8414286
epoch: 8620 정확도:  0.7757143
epoch: 8621 정확도:  0.7814286
epoch: 8622 정확도:  0.8242857
epoch: 8623 정확도:  0.83285713
epoch: 8624 정확도:  0.8357143
epoch: 8625 정확도:  0.8385714
epoch: 8626 정확도:  0.85142857
epoch: 8627 정확도:  0.85
epoch: 8628 정확도:  0.86142856
epoch: 8629 정확도:  0.8157143
epoch: 8630 정확도:  0.8157143
epoch: 8631 정확도:  0.8

epoch: 8922 정확도:  0.83285713
epoch: 8923 정확도:  0.8357143
epoch: 8924 정확도:  0.84
epoch: 8925 정확도:  0.8414286
epoch: 8926 정확도:  0.85
epoch: 8927 정확도:  0.8585714
epoch: 8928 정확도:  0.8357143
epoch: 8929 정확도:  0.85571426
epoch: 8930 정확도:  0.8
epoch: 8931 정확도:  0.80857146
epoch: 8932 정확도:  0.8385714
epoch: 8933 정확도:  0.8457143
epoch: 8934 정확도:  0.8585714
epoch: 8935 정확도:  0.82285714
epoch: 8936 정확도:  0.83285713
epoch: 8937 정확도:  0.8471429
epoch: 8938 정확도:  0.8585714
epoch: 8939 정확도:  0.82714283
epoch: 8940 정확도:  0.8314286
epoch: 8941 정확도:  0.8157143
epoch: 8942 정확도:  0.83428574
epoch: 8943 정확도:  0.79714286
epoch: 8944 정확도:  0.8042857
epoch: 8945 정확도:  0.8371429
epoch: 8946 정확도:  0.8357143
epoch: 8947 정확도:  0.8471429
epoch: 8948 정확도:  0.8428571
epoch: 8949 정확도:  0.85142857
epoch: 8950 정확도:  0.7985714
epoch: 8951 정확도:  0.81142855
epoch: 8952 정확도:  0.8385714
epoch: 8953 정확도:  0.8471429
epoch: 8954 정확도:  0.85714287
epoch: 8955 정확도:  0.8385714
epoch: 8956 정확도:  0.8442857
epoch: 8957 정확도:  0.77714

In [546]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    Z = logits.eval(feed_dict={X: test})
    y_pred = np.argmax(Z, axis=1)
    
np.mean(y_pred == test_labels.values)

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


0.81

In [547]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(solver='liblinear')

log_reg.fit(train.values, train_labels.values.astype(np.int))

pred = log_reg.predict(train.values)
np.mean(pred == train_labels.values) ## in-sample

0.96

In [548]:
pred = log_reg.predict(test.values)
np.mean(pred == test_labels.values)

0.9133333333333333

### c.	기준확률 = 0.5일 때 혼동행렬을 계산여라.  

In [572]:
sigm = 1. / (1. + np.exp(-Z[:,1]))
sigm2 = np.array(sigm > 0.5)

from sklearn.metrics import confusion_matrix

confm = confusion_matrix(test_labels.values.astype(int), sigm2)
confm

array([[193,  34],
       [ 23,  50]], dtype=int64)

### d.	기준확률 = 0.5일 때 정확도, 민감도, 특이도, 정밀도를 계산하여라  

In [577]:
print("정확도: ", (confm[0,0]+confm[1,1])/(confm[0,0]+confm[0,1]+confm[1,0]+confm[1,1]))
print("민감도: ", confm[1,1]/(confm[0,1]+confm[1,1]))
print("특이도: ", confm[0,0]/(confm[0,0]+confm[0,1]))
print("정밀도: ", confm[1,1]/(confm[0,1]+confm[1,1]))

정확도:  0.81
민감도:  0.5952380952380952
특이도:  0.8502202643171806
정밀도:  0.5952380952380952


머신러닝 logistic

In [580]:
from sklearn.metrics import confusion_matrix

confm = confusion_matrix(test_labels.values.astype(int), pred)
confm

array([[216,  11],
       [ 15,  58]], dtype=int64)

In [581]:
print("정확도: ", (confm[0,0]+confm[1,1])/(confm[0,0]+confm[0,1]+confm[1,0]+confm[1,1]))
print("민감도: ", confm[1,1]/(confm[0,1]+confm[1,1]))
print("특이도: ", confm[0,0]/(confm[0,0]+confm[0,1]))
print("정밀도: ", confm[1,1]/(confm[0,1]+confm[1,1]))

정확도:  0.9133333333333333
민감도:  0.8405797101449275
특이도:  0.9515418502202643
정밀도:  0.8405797101449275
